In [ ]:
import pandas as pd

matches_train_data = pd.read_csv('../data/raw/matches.csv')

player_data_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in range(1, 6) 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in ["Id", "Name", "Rating"]
                for side in ["Both"]]

df_matches_train_data = matches_train_data[player_data_columns]

prefixes = ['map{}Team{}SideBothPlayer{}']

rows = []

def process_player(player_id, player_name, player_rating):
    if player_id == "Not Available" or player_name == "Not Available" or player_rating == "Not Available":
        return
    else:
        player_rating = float(player_rating)

    rows.append({'id': int(player_id), 'name': str(player_name).strip(), 'rating': float(player_rating)})

for map_num in range(1, 6):
    for team_num in range(1, 3):
        for player_num in range(1, 6):
            player_id_col = f'map{map_num}Team{team_num}SideBothPlayer{player_num}Id'
            player_name_col = f'map{map_num}Team{team_num}SideBothPlayer{player_num}Name'
            player_rating_col = f'map{map_num}Team{team_num}SideBothPlayer{player_num}Rating'
            
            if player_id_col in df_matches_train_data.columns:
                for idx, player_id in df_matches_train_data[player_id_col].items():
                    player_name = df_matches_train_data.at[idx, player_name_col]
                    player_rating = df_matches_train_data.at[idx, player_rating_col]
                    process_player(player_id, player_name, player_rating)

all_ratings_df = pd.DataFrame(rows)

PROCESSED_FILES_FOLDER = '../data/processed'
OUTPUT_PATH = f'{PROCESSED_FILES_FOLDER}/all_ratings.csv'

all_ratings_df.to_csv(OUTPUT_PATH, index=False)

aggregated_df = all_ratings_df.groupby(['id']).agg(
    name=('name', 'first'),
    min_rating=('rating', 'min'),
    avg_rating=('rating', 'mean'),
    max_rating=('rating', 'max'),
    matches_count=('id', 'size')
).reset_index()

display(aggregated_df)

max_indices = aggregated_df['max_rating'].idxmax()
min_indices = aggregated_df['max_rating'].idxmin()

max_rows = aggregated_df.loc[max_indices]
min_rows = aggregated_df.loc[min_indices]

display(max_rows)
display(min_rows)

In [ ]:
# Definindo o critério para os melhores jogadores
n_players = 10
min_avg_rating_threshold = 1.2
min_matches_played_threshold = aggregated_df['matches_count'].quantile(0.90)

best_players = aggregated_df[aggregated_df['avg_rating'] > min_avg_rating_threshold]
best_players = aggregated_df[aggregated_df['matches_count'] > min_matches_played_threshold]
best_players = best_players.sort_values(by='avg_rating', ascending=False)
best_players = best_players[:n_players]
best_players = best_players.reset_index(drop=True)
best_players

In [ ]:

import matplotlib.pyplot as plt
sorted_df = best_players.sort_values(by='avg_rating', ascending=True)

plt.figure(figsize=(10, 6))
ax = sorted_df.plot(kind='bar', x='name', y=['min_rating', 'avg_rating', 'max_rating'], figsize=(10, 6), title="Ratings dos Jogadores", xlabel="Jogadores", ylabel="Ratings", legend=True)
plt.xticks(rotation=75)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
sorted_df = best_players.sort_values(by='avg_rating', ascending=True)
sorted_df['avg_rating'] = sorted_df['avg_rating'].round(2)

# Criar o scatter plot para avg_rating e max_rating
plt.figure(figsize=(10, 6))
plt.scatter(sorted_df['name'], sorted_df['avg_rating'], color='blue', label='Avg Rating')
plt.scatter(sorted_df['name'], sorted_df['max_rating'], color='red', label='Max Rating')

# Adicionar título e rótulos
plt.title('Scatter Plot de Ratings dos Jogadores')
plt.xlabel('Jogadores')
plt.ylabel('Rating')

# Adicionar rótulos aos pontos
for i in range(len(sorted_df)):
    plt.text(sorted_df['name'].iloc[i], sorted_df['avg_rating'].iloc[i], f'{sorted_df["avg_rating"].iloc[i]}', fontsize=9, ha='right', color='blue')
    plt.text(sorted_df['name'].iloc[i], sorted_df['max_rating'].iloc[i], f'{sorted_df["max_rating"].iloc[i]}', fontsize=9, ha='left', color='red')

plt.xticks(rotation=75)
plt.legend()
plt.show()